In [47]:
import pymysql
from math import *
import numpy as np

In [2]:
connection = pymysql.connect(host='127.0.0.1',
                             user='dbproject',
                             password='tycjytzp',
                             db='ShanghaiOsm',
                             charset='utf8mb4',
                             port=3306,
                             cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

In [3]:
cursor.execute("SELECT NodeID, Lat, Lon from Node")
Node2Pos = {i['NodeID']: (i['Lat'], i['Lon']) for i in cursor.fetchall()}

In [9]:
Way2Nodes = {}
cursor.execute("SELECT WayID, NodeID, OrderNum from WayNode")
for i in cursor.fetchall():
    if i['WayID'] not in Way2Nodes:
        Way2Nodes[i['WayID']] = []
    Way2Nodes[i['WayID']].append((i['OrderNum'], i['NodeID']))


In [11]:
for v in Way2Nodes.values():
    v.sort()

In [4]:
Node2Pos[1]

(31.226864, 121.531082)

In [13]:
Way2Nodes[4531289]

[(0, 28111346),
 (1, 28111345),
 (2, 28111357),
 (3, 28111354),
 (4, 28111352),
 (5, 28111350),
 (6, 28111348),
 (7, 28111346)]

In [37]:
dist = {k:[] for k in Way2Nodes.keys()}
R = 6371.004
lon_base = 95249
lat_base = 111122
for k, v in Way2Nodes.items():
    for i in range(len(v) - 1):
        lat_diff = (Node2Pos[v[i][1]][0] - Node2Pos[v[i + 1][1]][0]) * lat_base
        lon_diff = (Node2Pos[v[i][1]][1] - Node2Pos[v[i + 1][1]][1]) * lon_base
        dist[k].append(sqrt(lat_diff * lat_diff + lon_diff * lon_diff))
        lat0 = (90 - Node2Pos[v[i][1]][0]) / 180 * pi
        lat1 = (90 - Node2Pos[v[i + 1][1]][0]) / 180 * pi
        lon0 = Node2Pos[v[i][1]][1] / 180 * pi
        lon1 = Node2Pos[v[i + 1][1]][1] / 180 * pi
        C = cos(lat0)*cos(lat1)*cos(lon0-lon1) + sin(lat0)*sin(lat1)
        if C > 1:
            if abs(C - 1) < 1e-6:
                C = 1.0
        dist[k].append(R*acos(C))

In [38]:
dist[4531289]

[56.6802236847368,
 0.05671225630071251,
 58.31050875878085,
 0.03716910205916816,
 20.317756794765955,
 0.016810261427434057,
 59.09525382861307,
 0.05829773073278821,
 113.39532300532477,
 0.06815948773246328,
 27.796823873519568,
 0.027804717054505386,
 28.57470000094858,
 0.017162569278233262]

In [45]:
dist_list = []
for k, v in dist.items():
    if v != []:
        if max(v) > 10000:
            print(k, v)
    dist_list += v

364661234 [5210.897739507702, 3.127409546372536, 2880.6004065112575, 1.7253269751523526, 4413.2280103128, 2.67793433788023, 495.58054700083136, 0.2965695024396498, 748.9561181760797, 0.5331419738196318, 629.9066473493209, 0.480887845362843, 2585.511024822116, 2.45155318636489, 2585.3106092281996, 1.7651365733559723, 1913.3753592095845, 1.8434845537445632, 745.1758978680962, 0.7043014951849508, 26506.186601235353, 26.464638049887302, 15849.155036143642, 14.09187283585226, 8372.732745217094, 7.460623604316777, 7884.261269437034, 6.769480664386991, 7755.521670028234, 5.848429822096241, 13302.019007720528, 9.12833646045556]
27303455 [5741.065228297647, 3.5283198913686524, 11261.40371591458, 7.273195377264711, 9688.614258082756, 6.4804413230329585, 7154.2046635703255, 5.1900782292663195, 8167.447328714667, 5.767659951902887, 4703.2356858241765, 3.109503996835616, 8391.061837006933, 5.224181814487293]
319101439 [17913.5754553274, 16.857410500672696, 5098.396826013611, 3.4277400715466824, 831

In [41]:
len(dist_list)

2344322

In [42]:
max(dist_list)

36821.518275229115

In [46]:
num = 0
for i in dist_list:
    num += i // 100
num

580955.0

In [52]:
dists = np.array([list(i) for i in Node2Pos.values()])

In [54]:
dists.shape

(1913973, 2)

In [55]:
dists[:, 0].max()

31.787997000000001

In [56]:
dists[:, 0].min()

30.614000999999998